In [ ]:
import json

from glob import glob
import os.path

main_folder = '/home/schirrmr/data/auto-diagnosis/tuh-eeg/reports/edf/'


In [ ]:
import re
def extract_dict(filetext):
    filetext = filetext.replace('\t', '')
    lines = filetext.split('\n')
    current_tag = None
    current_text = ''
    tag_to_text = dict()
    for line in lines:
        new_tag_starts = re.match('[A-Z \t]+:', line)
        if new_tag_starts:
            if current_tag is not None:
                tag_to_text[current_tag] = current_text
            current_tag =  new_tag_starts.group(0)[:-1]
            current_text = line[len(current_tag) + 1:]
            current_tag = current_tag.strip()
        else:
            current_text += line
        assert current_tag is not None
    tag_to_text[current_tag] = current_text
    return tag_to_text
    

In [ ]:
%%time
filenames = glob(os.path.join(main_folder , '**/*.txt'), recursive=True)

In [ ]:
%%time

n_parse_failure = 0
n_unicode_failure = 0
all_dicts = []
for i_filename, filename in enumerate(filenames):
    if i_filename % 100 == 0:
        print("Processing file {:d} of {:d}, {:.2f}%...".format(i_filename + 1,
                                                               len(filenames),
                                                               (100 * i_filename) / float(len(filenames))))
    
    try:
        filetext = open(filename, 'r').read()
    except UnicodeDecodeError:
        print("Unicode failure")
        n_unicode_failure += 1
        continue
    try:
        report_dict = extract_dict(filetext)
    except AssertionError:
        print("Failure with {:s}".format(filetext))
        n_parse_failure += 1
        continue
    all_dicts.append(report_dict)

In [ ]:
import functools

In [ ]:
keys = [set(d.keys()) for d in all_dicts]

In [ ]:
set.union(*keys)

In [ ]:
all_file_texts = []
for i_filename, filename in enumerate(filenames):
    if i_filename % 100 == 0:
        print("Processing file {:d} of {:d}, {:.1f}%...".format(i_filename + 1,
                                                               len(filenames),
                                                               (100 * i_filename) / float(len(filenames))))
    filetext= open(filename, 'r', encoding='iso-8859-1').read()
    all_file_texts.append(filetext)
    #try:
    #    filetext = open(filename, 'r').read()
    #except UnicodeDecodeError:
    #    filetext= open(filename, 'r', encoding='iso-8859-1')
    #    n_unicode_failure += 1
    #    continue
    

In [ ]:
all_dicts[0]

In [ ]:
all_dicts = [dict(text=text, filename=filename.replace(main_folder,''), nr=nr)
             for nr, filename, text in
      zip(range(len(all_file_texts)), filenames, all_file_texts)]

In [ ]:
json.dump(all_dicts[:100], open('auto-eeg-data-small.json', 'w'))

In [ ]:
json.dump(all_dicts, open('auto-eeg-data.json', 'w'))

In [ ]:
cat auto-eeg-data.json